<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/model-template/MachineHack_ELECTRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 3.0MB 17.8MB/s 
     |████████████████████████████████| 890kB 45.5MB/s 
     |████████████████████████████████| 1.1MB 40.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=43aaa99268fccfcb8af1102988b9c5300753708c5e29b957cb55ae84419c171b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
!unzip ./Embold_Participant\'s_Dataset.zip -d Dataset

--2020-10-14 12:23:33--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.62.64
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.62.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  13.1MB/s    in 9.1s    

2020-10-14 12:23:43 (10.7 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Dataset/Embold_Participant's_Dataset/
  inflating: Dataset/Embold_Participant's_Dataset/sample submission.csv  
  inflating: Dataset/__MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Dataset/Embold_Participant's_Dataset/embold_train_extra

In [ ]:
cd "Dataset/Embold_Participant's_Dataset/"

/content/Dataset/Embold_Participant's_Dataset


In [ ]:
#import pandas as pd
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
train_ex_df = pd.read_json("embold_train_extra.json")

In [ ]:
test_label_df = pd.read_csv('/content/level3ensemble_v6.csv')
test_df['label'] = test_label_df['label']

In [ ]:
train_data = train_df.append(train_ex_df)
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

In [ ]:
train_data.shape

(450000, 4)

In [ ]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import *
from transformers import pipeline
from pprint import pprint
from tensorflow import keras

In [ ]:
@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "google/electra-small-discriminator"  # huggingface bert model


transformer_flags = Config()


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = transformer_flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.16.148.202:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.16.148.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.16.148.202:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

from transformers import *
#tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = AlbertTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
#tokenizer = RobertaTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
tokenizer = ElectraTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

In [ ]:
#tf.keras.backend.clear_session()

In [ ]:
def build_model():
  input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
  bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL)
  #bert_model = TFAutoModel.from_pretrained('/content/github-albert-xlarge-v2/')
  #bert_model = TFAutoModel.from_pretrained('../../github-roberta-large/')
  output_layer = bert_model(input_layer) 
  pooled_output = tf.keras.layers.GlobalAveragePooling1D()(output_layer[0])
  output = tf.keras.layers.Dense(3, activation='softmax')(pooled_output)
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam(learning_rate=transformer_flags.LR)
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()
    
  return classifier_model

In [ ]:
all_train = train_data[0:200000]
all_train = all_train.append(train_data[230000:])

In [ ]:
question_df = all_train[all_train.label==2]
bug_issue = all_train[all_train.label!=2]
bug_issue = bug_issue.sample(150000)
train = question_df.append(bug_issue)
train = train.append(test_df)
train = train.sample(len(train))

In [ ]:
train.label.value_counts()

1    90245
0    87088
2    41992
Name: label, dtype: int64

In [ ]:
validate = train_data[200000:230000]
validate.label.value_counts()

1    13846
0    13278
2     2876
Name: label, dtype: int64

In [ ]:
with strategy.scope():
    classifier_model = build_model()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the model checkpoint at google/electra-small-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
tf_electra_model (TFElectraM ((None, 320, 256),)       13483008  
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 771       
Total params: 13,483,779
Trainable params: 13,483,779
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# data preparation
X_data = regular_encode(train.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_train = train.label.values
print(X_data.shape, y_train.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(219325, 320) (219325,)


In [ ]:
# data preparation
X_val = regular_encode(validate.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_val = validate.label.values
print(X_val.shape, y_val.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(30000, 320) (30000,)


In [ ]:
X_test = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=2,batch_size=128, validation_data=(X_val, y_val))

Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/1714 [..............................] - ETA: 7:39:57 - loss: 1.0667 - accuracy: 0.4492 WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0104s vs `on_train_batch_end` time: 0.0938s). Check your callbacks.


1714/1714 [==============================] - ETA: 0s - loss: 0.6056 - accuracy: 0.7547WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0049s vs `on_test_batch_end` time: 0.0333s). Check your callbacks.


1714/1714 [==============================] - 256s 149ms/step - loss: 0.6056 - accuracy: 0.7547 - val_loss: 0.4948 - val_accuracy: 0.8109
Epoch 2/2
1714/1714 [==============================] - 184s 107ms/step - loss: 0.5091 - accuracy: 0.8057 - val_loss: 0.4868 - val_accuracy: 0.8140


In [ ]:
# 230000:430000 datapoint
#classifier_model.fit(X_data, y_train, epochs=1,batch_size=128, validation_data=(X_val, y_val))

   2/1309 [..............................] - ETA: 2:10 - loss: 0.5266 - accuracy: 0.7930WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0098s vs `on_train_batch_end` time: 0.0976s). Check your callbacks.


1309/1309 [==============================] - ETA: 0s - loss: 0.5045 - accuracy: 0.8067WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_test_batch_end` time: 0.0323s). Check your callbacks.


1309/1309 [==============================] - 146s 111ms/step - loss: 0.5045 - accuracy: 0.8067 - val_loss: 0.4953 - val_accuracy: 0.8102


In [ ]:
y_val_pred = classifier_model.predict(X_val, batch_size=global_batch_size, verbose=1)
y_test_pred = classifier_model.predict(X_test, batch_size=global_batch_size, verbose=1)

  2/235 [..............................] - ETA: 11:22WARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0035s vs `on_predict_batch_end` time: 0.0385s). Check your callbacks.


  1/235 [..............................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0032s vs `on_predict_batch_end` time: 0.0306s). Check your callbacks.


235/235 [==============================] - 8s 34ms/step


In [ ]:
eval_df = pd.DataFrame({'X':y_val_pred.tolist(), 'y':y_val.tolist()})

In [ ]:
eval_df.to_csv('electra-small-maxlen320-PL-2epochs-90K87K40K-eval-prob.csv', index=False)

In [ ]:
submission = pd.read_csv('sample submission.csv')
submission['label'] = y_test_pred.tolist()
file_name = 'electra-small-maxlen320-PL-2epochs-90K87K40K-test-prob.csv'
submission.to_csv(file_name, index=False)